<a href="https://colab.research.google.com/github/SaadTazroute/JurisprudencesClassification/blob/master/02%20TF-IDF%20%2B%20ML%20ALGORITHMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:

plt.rcParams['font.size'] = 14
np.set_printoptions(precision=3)
# better display of review text in dataframes
pd.set_option('display.max_colwidth', None) 

# Seaborn options
sns.set(style="whitegrid", font_scale=1.4)

# Auto reload modules
%load_ext autoreload
%autoreload 2

In [ ]:
import re
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Prepare data

## Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path='/content/drive/My Drive/Freemium02.csv'
import pandas as pd
data_df = pd.read_csv(path, error_bad_lines=False)

data_df = data_df.reindex(np.random.permutation(data_df.index))
data_df.reset_index(inplace=True, drop=True)

data_df=data_df.dropna(how = 'any')

data_df=data_df[:10000]
data_df = data_df.reindex(np.random.permutation(data_df.index))
data_df.reset_index(inplace=True, drop=True)

#data_df = data_df.drop('idx', 1)

data_df.head(2)



data_df = pd.DataFrame({'content':data_df['content'],'solution':data_df['solution']})
data_df['solution']=data_df['solution'].str.lower()
data_df







content   solution
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
resultat = data_df.groupby('solution')['content'].nunique()
resultat

solution
annulation                                      21
cassation                                     4356
irrecevabilite                                 228
nonlieu statuer                                  8
qpc nonlieu renvoi conseil constitutionnel      19
rejet                                         5368
Name: content, dtype: int64

In [ ]:
#we try to reduce the lenght of text : approche utilisé  : troncature du texte au milieu et prendre un nombre N  de mots: A revoir l'article 

def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[250:500]
      l_total.append(" ".join(l_parcial))
  return l_total




In [ ]:
data_df['content'] = data_df['content'].apply(get_split)
data_df['content']=data_df['content'].astype(str)

data_df.head()


,content,solution
0,"[""; signé par deux représentants de la société Kalamazoo prévoit sous le titre & quot ; Volumes & quot ; : 5 représentants , 600 factures , 500 bons de dépôt et 1000 clients ; qu ' un éventuel dépassement des prévisions contractuelles supposait donc un traitement mensuel de plus de 600 factures et de plus de 500 bons de dépôts , l ' existence de représentants supplémentaires et le franchissement du seuil des 1200 clients , dépassement qui n ' a nullement été constaté par la cour d ' appel ; qu ' en ne constatant pas le dépassement des prévisions contractuelles et en retenant néanmoins un défaut d ' information de la part de l ' acquéreur , la cour d ' appel a privé sa décision de base légale au regard de l ' article 1147 du Code civil ; Mais attendu qu ' après avoir constaté que l ' équipement cédé était adapté aux besoins de la société FSD à la date du contrat et qu ' il n ' est devenu insuffisant que du fait de l ' augmentation importante et rapide du chiffre d ' affaires de celle - ci et de l ' accroissement corrélatif du volume des opérations à traiter sur le plan comptable , la cour d ' appel a énoncé à bon droit que les manquements éventuels du fournisseur d ' un équipement informatique à son devoir de conseil , dont elle a relevé exactement qu ' il se limitait""]",rejet
1,"['anciens horaires ; qu \' il a été licencié pour faute grave par lettre du 2 juillet 2004 ; que , contestant le bien - fondé de son licenciement , le salarié a saisi la juridiction prud \' homale ;< br clear ="" none ""/> < br clear ="" none ""/> Attendu que pour dire le licenciement de M . X ... fondé sur une cause réelle et sérieuse , la cour d \' appel retient que le salarié ne se prévalait pas d \' une clause contractuelle excluant le travail les samedi et dimanche et que la modification des jours et horaires de travail décidée par l \' employeur était justifiée par des impératifs de fonctionnement ;< br clear ="" none ""/> < br clear ="" none ""/> Qu \' en statuant ainsi , alors que la nouvelle répartition de l \' horaire de travail avait pour effet de priver le salarié du repos dominical , ce qui constituait une modification de son contrat de travail qu \' il était en droit de refuser , la cour d \' appel a violé le texte susvisé ;< br clear ="" none ""/> < br clear ="" none ""/> PAR CES MOTIFS :< br clear ="" none ""/> < br clear ="" none ""/> CASSE ET ANNULE , mais seulement en ce qu \' il a dit le licenciement de M . X ... fondé sur une cause réelle et sérieuse , l \' arrêt rendu le 10 septembre 2009 ,']",cassation
2,"[""IL N ' EN SERAIT PAS AINSI , EN L ' ESPECE , OU LEDIT DOMMAGE AVAIT ETE CAUSE LORS DE LA CONDUITE DE L ' ANIMAL , CELUI - CI RESTANT A CE MOMENT SOUS LA GARDE DE SON PROPRIETAIRE ; MAIS ATTENDU QU ' APRES AVOIR PRIS LIVRAISON DU CHEVAL EN VUE DE LE CONDUIRE DANS SON ATELIER , POUR Y ACCOMPLIR UN ACTE DE SON METIER , VESIR AVAIT DESORMAIS L ' USAGE , LA DIRECTION ET LE CONTROLE DE L ' ANIMAL , ET AGISSAIT EN QUALITE DE MARECHAL - FERRANT INDEPENDANT ET DANS LE SEUL INTERET DE L ' EXERCICE DE SA PROFESSION ; D ' OU IL SUIT QUE LA COUR D ' APPEL , EN DECLARANT QUE LA VILLE DE CAHORS , AVAIT CONSERVE LA GARDE DE L ' ANIMAL , VESIR ETANT EN L ' ESPECE SON PREPOSE , SANS INDIQUER EN QUOI UN LIEN DE SUBORDINATION AURAIT EXISTE ENTRE EUX , A AINSI FAUSSEMENT APPLIQUE ET , PARTANT , VIOLE LE TEXTE SUSVISE ; PAR CES MOTIFS : ET SANS QU ' IL Y AIT LIEU DE STATUER SUR LE SECOND MOYEN : CASSE ET ANNULE L ' ARRET RENDU ENTRE LES PARTIES PAR LA COUR D ' APPEL D ' AGEN , LE 26 MARS 1958 ; REMET , EN CONSEQUENCE , LA CAUSE ET LES PARTIES AU MEME ET SEMBLABLE ETAT OU ELLES ETAIENT AVANT LEDIT ARRET ET , POUR ETRE FAIT DROIT , LES RENVOIE DEVANT LA COUR""]",cassation
3,"['br clear ="" none ""/> Sur la recevabilité du mémoire personnel produit en défense :< br clear ="" none ""/> < br clear ="" none ""/> Attendu que ce mémoire n \' est pas signé par un avocat à la Cour de cassation ; que , dès lors , il est irrecevable , p

## Text Preprocessing & Cleaning


In [ ]:
# Load NLTK stop words
import nltk
nltk.download('stopwords')
stop_words_fr = stopwords.words('french')
stop_words_fr = set(stop_words_fr)
len(stop_words_fr)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


157

In [ ]:
#we can also add an entire list of words

L = ['\n', ' ', '!', '"', '#', '$', '%',  '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '\x80', '\x84', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x9c', '¡', '£', '§', '¨', '«', '¬', '®', '°', '²', '³', 'µ', '·', '¸', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Â', 'Ä', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Î', 'Ï', 'Ó', 'Ô', 'Ö', '×', 'Ù', 'Û', 'Ü', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ù', 'ú', 'û', 'ü', 'ÿ', 'œ', 'š', 'Ÿ', '–', '—', '‘', '’', '„', '•', '…', '€' ,'celle', 'celles', 'certain', 'certains', 'les','ils', 'elles', 'avoir', 'être', 'parce','où','si','chaque','donc','dont','encore','faut','quand','tant','tel','va','ici', 'ça']
L=set(L)
stop_words_fr = stop_words_fr | L
stop_words_fr

{'\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avoir',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'celle',
 'celles',
 'certain',
 'certains',
 'ces',
 'chaque',
 'd',
 'dans',
 'de',
 'des',
 'donc',
 'dont',
 'du',
 'elle',
 'elles',
 'en',
 'encore',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'faut',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 '

In [ ]:
# convert to lowercase and tokenize
tokenizer = RegexpTokenizer('[^_\W0-9]+')
corpus_tokens_ = [tokenizer.tokenize(content.lower()) for content in data_df.content]
data_df['tokens'] = corpus_tokens_

In [ ]:
# Je me suis basé sur le notebook de l'analyse des jurisprudences pour détecter les mots à supprimer + stopwords 

# Finally remove stop words from our corpus
u = [[word for word in sent if word not in stop_words_fr] for sent in data_df.tokens]
assert (len(u) == len(data_df))
data_df.tokens = pd.Series(u)
data_df.head()

,content,solution,tokens
0,"[""; signé par deux représentants de la société Kalamazoo prévoit sous le titre & quot ; Volumes & quot ; : 5 représentants , 600 factures , 500 bons de dépôt et 1000 clients ; qu ' un éventuel dépassement des prévisions contractuelles supposait donc un traitement mensuel de plus de 600 factures et de plus de 500 bons de dépôts , l ' existence de représentants supplémentaires et le franchissement du seuil des 1200 clients , dépassement qui n ' a nullement été constaté par la cour d ' appel ; qu ' en ne constatant pas le dépassement des prévisions contractuelles et en retenant néanmoins un défaut d ' information de la part de l ' acquéreur , la cour d ' appel a privé sa décision de base légale au regard de l ' article 1147 du Code civil ; Mais attendu qu ' après avoir constaté que l ' équipement cédé était adapté aux besoins de la société FSD à la date du contrat et qu ' il n ' est devenu insuffisant que du fait de l ' augmentation importante et rapide du chiffre d ' affaires de celle - ci et de l ' accroissement corrélatif du volume des opérations à traiter sur le plan comptable , la cour d ' appel a énoncé à bon droit que les manquements éventuels du fournisseur d ' un équipement informatique à son devoir de conseil , dont elle a relevé exactement qu ' il se limitait""]",rejet,"[signé, deux, représentants, société, kalamazoo, prévoit, sous, titre, quot, volumes, quot, représentants, factures, bons, dépôt, clients, éventuel, dépassement, prévisions, contractuelles, supposait, traitement, mensuel, plus, factures, plus, bons, dépôts, existence, représentants, supplémentaires, franchissement, seuil, clients, dépassement, a, nullement, constaté, cour, appel, constatant, dépassement, prévisions, contractuelles, retenant, néanmoins, défaut, information, part, acquéreur, cour, appel, a, privé, décision, base, légale, regard, article, code, civil, attendu, après, constaté, équipement, cédé, adapté, besoins, société, fsd, date, contrat, devenu, insuffisant, fait, augmentation, importante, rapide, chiffre, affaires, ci, accroissement, corrélatif, volume, opérations, traiter, plan, comptable, cour, appel, a, énoncé, bon, droit, manquements, éventuels, fournisseur, équipement, informatique, devoir, ...]"
1,"['anciens horaires ; qu \' il a été licencié pour faute grave par lettre du 2 juillet 2004 ; que , contestant le bien - fondé de son licenciement , le salarié a saisi la juridiction prud \' homale ;< br clear ="" none ""/> < br clear ="" none ""/> Attendu que pour dire le licenciement de M . X ... fondé sur une cause réelle et sérieuse , la cour d \' appel retient que le salarié ne se prévalait pas d \' une clause contractuelle excluant le travail les samedi et dimanche et que la modification des jours et horaires de travail décidée par l \' employeur était justifiée par des impératifs de fonctionnement ;< br clear ="" none ""/> < br clear ="" none ""/> Qu \' en statuant ainsi , alors que la nouvelle répartition de l \' horaire de travail avait pour effet de priver le salarié du repos dominical , ce qui constituait une modification de son contrat de travail qu \' il était en droit de refuser , la cour d \' appel a violé le texte susvisé ;< br clear ="" none ""/> < br clear ="" none ""/> PAR CES MOTIFS :< br clear ="" none ""/> < br clear ="" none ""/> CASSE ET ANNULE , mais seulement en ce qu \' il a dit le licenciement de M . X ... fondé sur une cause réelle et sérieuse , l \' arrêt rendu le 10 septembre 2009 ,']",cassation,"[anciens, horaires, a, licencié, faute, grave, lettre, juillet, contestant, bien, fondé, licenciement, salarié, a, saisi, juridiction, prud, homale, br, clear, none, br, clear, none, attendu, dire, licenciement, x, fondé, cause, réelle, sérieuse, cour, appel, retient, salarié, prévalait, clause, contractuelle, excluant, travail, samedi, dimanche, modification, jours, horaires, travail, décidée, employeur, justifiée, impératifs, fonctionnement, br, clear, none, br, clear, none, statuant,

In [ ]:
#On remplace le texte original par le text qu'on a nettoyé.

L2 = [' '.join(doc) for doc in data_df.tokens]
data_df['content'] = L2
data_df.drop(['tokens'],axis=1,inplace=True)
data_df

,content,solution
0,signé deux représentants société kalamazoo prévoit sous titre quot volumes quot représentants factures bons dépôt clients éventuel dépassement prévisions contractuelles supposait traitement mensuel plus factures plus bons dépôts existence représentants supplémentaires franchissement seuil clients dépassement a nullement constaté cour appel constatant dépassement prévisions contractuelles retenant néanmoins défaut information part acquéreur cour appel a privé décision base légale regard article code civil attendu après constaté équipement cédé adapté besoins société fsd date contrat devenu insuffisant fait augmentation importante rapide chiffre affaires ci accroissement corrélatif volume opérations traiter plan comptable cour appel a énoncé bon droit manquements éventuels fournisseur équipement informatique devoir conseil a relevé exactement limitait,rejet
1,anciens horaires a licencié faute grave lettre juillet contestant bien fondé licenciement salarié a saisi juridiction prud homale br clear none br clear none attendu dire licenciement x fondé cause réelle sérieuse cour appel retient salarié prévalait clause contractuelle excluant travail samedi dimanche modification jours horaires travail décidée employeur justifiée impératifs fonctionnement br clear none br clear none statuant ainsi alors nouvelle répartition horaire travail effet priver salarié repos dominical constituait modification contrat travail droit refuser cour appel a violé texte susvisé br clear none br clear none motifs br clear none br clear none casse annule seulement a dit licenciement x fondé cause réelle sérieuse arrêt rendu septembre,cassation
2,ainsi espece ledit dommage ete cause lors conduite animal celui ci restant a moment sous garde proprietaire attendu apres pris livraison cheval vue conduire atelier accomplir acte metier vesir desormais usage direction controle animal agissait qualite marechal ferrant independant seul interet exercice profession suit cour appel declarant ville cahors conserve garde animal vesir etant espece prepose sans indiquer quoi lien subordination existe entre a ainsi faussement applique partant viole texte susvise motifs sans lieu statuer second moyen casse annule arret rendu entre parties cour appel agen mars remet consequence cause parties meme semblable etat etaient avant ledit arret etre fait droit renvoie devant cour,cassation
3,br clear none recevabilité mémoire personnel produit défense br clear none br clear none attendu mémoire signé avocat cour cassation dès lors irrecevable application article code procédure pénale br clear none br clear none moyen unique cassation pris violation article code route br clear none br clear none attendu résulte jugement attaqué pièces procédure juin véhicule automobile donné location société novartis pharma a fait objet procès verbal excès vitesse x désigné représentant légal cette société comme utilisateur habituel véhicule a cité comparaître devant juridiction proximité jugement attaqué a renvoyé fins poursuite retenant auteur infraction qualité représentant légal personne morale ni locataire véhicule,rejet
4,bien fonde demande possibilite determiner non personnellement subi prejudice direct pouvaient utilement invoquer autorite chose jugee puisque rien ete definitivement juge concerne quot alors part annulation arret generaux absolus termes lesquels a ete prononcee limitee a portee moyen a servi base laisse subsister comme passees force chose jugee toutes dispositions expresses implicites ete attaquees pourvoi resulte lors recevabilite constitution partie civile ete admise juge instruction confirmee jugement janvier non contestee arret casse cour appel dijon autorite chose jugee point quot alors autre part personnalite morale societe dissoute survit seuls besoins liquidation a dire mesure necessaire a accomplissement actes juridiques faciliteront liquidation partage,rejet
...,...,...
9995,prestations concouren a reparation prejudice corporel victime cour appel 

## Removal of Frequent words

In the previos preprocessing step, we removed the stopwords based on language information. But say, if we have a domain specific corpus, we might also have some frequent words which are of not so much importance to us. 

So this step is to remove the frequent words in the given corpus. If we use something like tfidf, this is automatically taken care of.  

Let us get the most common words adn then remove them in the next step


In [ ]:
from collections import Counter
cnt = Counter()
for text in data_df["content"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('a', 33992),
 ('cour', 12419),
 ('appel', 12095),
 ('attendu', 8364),
 ('article', 7708),
 ('quot', 7469),
 ('code', 6557),
 ('fait', 6504),
 ('x', 6448),
 ('moyen', 5504)]

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

data_df["content"] = data_df["content"].apply(lambda text: remove_freqwords(text))
data_df.head()

,content,solution
0,signé deux représentants société kalamazoo prévoit sous titre volumes représentants factures bons dépôt clients éventuel dépassement prévisions contractuelles supposait traitement mensuel plus factures plus bons dépôts existence représentants supplémentaires franchissement seuil clients dépassement nullement constaté constatant dépassement prévisions contractuelles retenant néanmoins défaut information part acquéreur privé décision base légale regard civil après constaté équipement cédé adapté besoins société fsd date contrat devenu insuffisant augmentation importante rapide chiffre affaires ci accroissement corrélatif volume opérations traiter plan comptable énoncé bon droit manquements éventuels fournisseur équipement informatique devoir conseil relevé exactement limitait,rejet
1,anciens horaires licencié faute grave lettre juillet contestant bien fondé licenciement salarié saisi juridiction prud homale br clear none br clear none dire licenciement fondé cause réelle sérieuse retient salarié prévalait clause contractuelle excluant travail samedi dimanche modification jours horaires travail décidée employeur justifiée impératifs fonctionnement br clear none br clear none statuant ainsi alors nouvelle répartition horaire travail effet priver salarié repos dominical constituait modification contrat travail droit refuser violé texte susvisé br clear none br clear none motifs br clear none br clear none casse annule seulement dit licenciement fondé cause réelle sérieuse arrêt rendu septembre,cassation
2,ainsi espece ledit dommage ete cause lors conduite animal celui ci restant moment sous garde proprietaire apres pris livraison cheval vue conduire atelier accomplir acte metier vesir desormais usage direction controle animal agissait qualite marechal ferrant independant seul interet exercice profession suit declarant ville cahors conserve garde animal vesir etant espece prepose sans indiquer quoi lien subordination existe entre ainsi faussement applique partant viole texte susvise motifs sans lieu statuer second casse annule arret rendu entre parties agen mars remet consequence cause parties meme semblable etat etaient avant ledit arret etre droit renvoie devant,cassation
3,br clear none recevabilité mémoire personnel produit défense br clear none br clear none mémoire signé avocat cassation dès lors irrecevable application procédure pénale br clear none br clear none unique cassation pris violation route br clear none br clear none résulte jugement attaqué pièces procédure juin véhicule automobile donné location société novartis pharma objet procès verbal excès vitesse désigné représentant légal cette société comme utilisateur habituel véhicule cité comparaître devant juridiction proximité jugement attaqué renvoyé fins poursuite retenant auteur infraction qualité représentant légal personne morale ni locataire véhicule,rejet
4,bien fonde demande possibilite determiner non personnellement subi prejudice direct pouvaient utilement invoquer autorite chose jugee puisque rien ete definitivement juge concerne alors part annulation arret generaux absolus termes lesquels ete prononcee limitee portee servi base laisse subsister comme passees force chose jugee toutes dispositions expresses implicites ete attaquees pourvoi resulte lors recevabilite constitution partie civile ete admise juge instruction confirmee jugement janvier non contestee arret casse dijon autorite chose jugee point alors autre part personnalite morale societe dissoute survit seuls besoins liquidation dire mesure necessaire accomplissement actes juridiques faciliteront liquidation partage,rejet


In [ ]:


seuil=0
data_df = data_df[data_df.groupby('solution').solution.transform('count')>seuil].copy() 
data_df
resultat = data_df.groupby('solution')['content'].nunique()
resultat


solution
annulation                                      21
cassation                                     4251
irrecevabilite                                 147
nonlieu statuer                                  5
qpc nonlieu renvoi conseil constitutionnel      19
rejet                                         5264
Name: content, dtype: int64

In [ ]:

# transformation of text labels : on peut utiliser labelencoder sur scikit learn aussi pour cette tache

data_df['category_id'] = data_df['solution'].factorize()[0]
category_id_df = data_df[['solution', 'category_id']].drop_duplicates()


# Dictionaries for future use
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'solution']].values)

data_df.head()

,content,solution,category_id
0,signé deux représentants société kalamazoo prévoit sous titre volumes représentants factures bons dépôt clients éventuel dépassement prévisions contractuelles supposait traitement mensuel plus factures plus bons dépôts existence représentants supplémentaires franchissement seuil clients dépassement nullement constaté constatant dépassement prévisions contractuelles retenant néanmoins défaut information part acquéreur privé décision base légale regard civil après constaté équipement cédé adapté besoins société fsd date contrat devenu insuffisant augmentation importante rapide chiffre affaires ci accroissement corrélatif volume opérations traiter plan comptable énoncé bon droit manquements éventuels fournisseur équipement informatique devoir conseil relevé exactement limitait,rejet,0
1,anciens horaires licencié faute grave lettre juillet contestant bien fondé licenciement salarié saisi juridiction prud homale br clear none br clear none dire licenciement fondé cause réelle sérieuse retient salarié prévalait clause contractuelle excluant travail samedi dimanche modification jours horaires travail décidée employeur justifiée impératifs fonctionnement br clear none br clear none statuant ainsi alors nouvelle répartition horaire travail effet priver salarié repos dominical constituait modification contrat travail droit refuser violé texte susvisé br clear none br clear none motifs br clear none br clear none casse annule seulement dit licenciement fondé cause réelle sérieuse arrêt rendu septembre,cassation,1
2,ainsi espece ledit dommage ete cause lors conduite animal celui ci restant moment sous garde proprietaire apres pris livraison cheval vue conduire atelier accomplir acte metier vesir desormais usage direction controle animal agissait qualite marechal ferrant independant seul interet exercice profession suit declarant ville cahors conserve garde animal vesir etant espece prepose sans indiquer quoi lien subordination existe entre ainsi faussement applique partant viole texte susvise motifs sans lieu statuer second casse annule arret rendu entre parties agen mars remet consequence cause parties meme semblable etat etaient avant ledit arret etre droit renvoie devant,cassation,1
3,br clear none recevabilité mémoire personnel produit défense br clear none br clear none mémoire signé avocat cassation dès lors irrecevable application procédure pénale br clear none br clear none unique cassation pris violation route br clear none br clear none résulte jugement attaqué pièces procédure juin véhicule automobile donné location société novartis pharma objet procès verbal excès vitesse désigné représentant légal cette société comme utilisateur habituel véhicule cité comparaître devant juridiction proximité jugement attaqué renvoyé fins poursuite retenant auteur infraction qualité représentant légal personne morale ni locataire véhicule,rejet,0
4,bien fonde demande possibilite determiner non personnellement subi prejudice direct pouvaient utilement invoquer autorite chose jugee puisque rien ete definitivement juge concerne alors part annulation arret generaux absolus termes lesquels ete prononcee limitee portee servi base laisse subsister comme passees force chose jugee toutes dispositions expresses implicites ete attaquees pourvoi resulte lors recevabilite constitution partie civile ete admise juge instruction confirmee jugement janvier non contestee arret casse dijon autorite chose jugee point alors autre part personnalite morale societe dissoute survit seuls besoins liquidation dire mesure necessaire accomplissement actes juridiques faciliteront liquidation partage,rejet,0


## TFIDF

The text needs to be transformed to vectors so as the algorithms will be able make predictions. In this case it will be used the Term Frequency – Inverse Document Frequency (TFIDF) weight to evaluate __how important a word is to a document in a collection of documents__.

After removing __punctuation__ and __lower casing__ the words, importance of a word is determined in terms of its frequency.




## TF-IDF

****### “Term Frequency – Inverse Document Frequency 

__TF-IDF__ is the product of the __TF__ and __IDF__ scores of the term.<br><br> $$\text{TF-IDF}=\frac{\text{TF}}{\text{IDF}}$$<br>

__Term Frequency :__ This summarizes how often a given word appears within a document.

$$\text{TF} = \frac{\text{Number of times the term appears in the doc}}{\text{Total number of words in the doc}}$$<br><br>
__Inverse Document Frequency:__ This downscales words that appear a lot across documents. A term has a high IDF score if it appears in a few documents. Conversely, if the term is very common among documents (i.e., “the”, “a”, “is”), the term would have a low IDF score.<br>

$$\text{IDF} = \ln\left(\frac{\text{Number of docs}}{\text{Number docs the term appears in}} \right)$$<br>


TF-IDF are word frequency scores that try to highlight words that are more interesting, e.g. frequent in a document but not across documents. The higher the TFIDF score, the rarer the term is. 


TfidfVectorizer class can be initialized with the following parameters:
* __min_df__: remove the words from the vocabulary which have occurred in less than ‘min_df’ number of files.
* __max_df__: remove the words from the vocabulary which have occurred in more than _‘max_df’ * total number of files in corpus_.
* __sublinear_tf__: set to True to scale the term frequency in logarithmic scale.
* __stop_words__: remove the predefined stop words in 'english'.
* __use_idf__: weight factor must use inverse document frequency.
* __ngram_range__: (1, 2) to indicate that unigrams and bigrams will be considered.

In [ ]:




tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2))

# We transform each text into a vector
features = tfidf.fit_transform(data_df.content).toarray()

labels = data_df.category_id

print("Chacune des %d  jurisprudences est représentée par %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))







    
    
    


Chacune des 10000  jurisprudences est représentée par 24056 features (TF-IDF score of unigrams and bigrams)


In [ ]:
# Finding the N most correlated terms with each of the product categories
N = 1
for solution, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("\n==> %s:" %(solution))
  print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
  print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))
    
    
    
    


==> annulation:
  * Most Correlated Unigrams are: punit
  * Most Correlated Bigrams are: declare nulle

==> cassation:
  * Most Correlated Unigrams are: renvoie
  * Most Correlated Bigrams are: renvoie devant

==> irrecevabilite:
  * Most Correlated Unigrams are: irrecevable
  * Most Correlated Bigrams are: motifs declare

==> nonlieu statuer:
  * Most Correlated Unigrams are: chiffrer
  * Most Correlated Bigrams are: devenu sans

==> qpc nonlieu renvoi conseil constitutionnel:
  * Most Correlated Unigrams are: constitutionnel
  * Most Correlated Bigrams are: conseil constitutionnel

==> rejet:
  * Most Correlated Unigrams are: renvoie
  * Most Correlated Bigrams are: renvoie devant


## Multi-Classification models

The classification models evaluated are: 
* Random Forest
* Linear Support Vector Machine
* Multinomial Naive Bayes 
* Logistic Regression.

In [ ]:
X = data_df['content'] # Collection of documents
y = data_df['solution'] # Target or the labels we want to predict 

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

MODELS

In [ ]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

## Comparison of model performance

The best mean acuracy was obtained with LinearSVC.

In [ ]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

In [ ]:
## Model evaluation

In [ ]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(features, 
                                                               labels, 
                                                               data_df.index, test_size=0.25, 
                                                               random_state=1)
model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
# Classification report
print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred, 
                                    target_names= data_df['solution'].unique()))

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=category_id_df.solution.values, 
            yticklabels=category_id_df.solution.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title("CONFUSION MATRIX - LinearSVC\n", size=16);